In [1]:
#BPR
import numpy
import tensorflow as tf
import os
import random
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial
from tqdm import tqdm
%matplotlib inline

In [2]:
DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"
DATA_FOR_BPR = "data_for_BPR.csv"
SAVE_DIR = "../gamedata/train_result/"

In [ ]:
# #data preprocessing
# data = pd.read_csv(DATA_DIR+DATA_FILE)
# gameid = data['gameid'].drop_duplicates()
# gameid_list = []
# gameid_list = list(gameid)
# gameid2newid = {old:new+1 for new, old in enumerate(gameid_list)}
# data['new_game_id'] = data['gameid'].map(gameid2newid)

# events = data['eventname'].drop_duplicates()
# events_list = list(events)
# events2id = {old:new+1 for new,old in enumerate(events_list)}
# data['new_event_id'] = data['eventname'].map(events2id)

In [ ]:
#data.to_csv(DATA_DIR+'data_for_BPR.csv',index=False)

In [ ]:
#data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [ ]:
#data[data['new_game_id']==63]

In [3]:
def load_data(data_path):
    game_events = defaultdict(set)
    game_count = -1
    event_count = -1
    with open(data_path,'r') as f:
        header_line = next(f)
        for line in f.readlines():
            _, _, _, _, _, _, gameid, eventid = line.split(",")
            gameid = int(gameid)
            eventid = int(eventid)
            #print(u,i)
            game_events[gameid].add(eventid)
            game_count = max(gameid, game_count)
            event_count = max(eventid, event_count)
    print ("game_count:", game_count)
    print ("event_count:", event_count)
    #return max_u_id, max_i_id, user_ratings
    return game_count,event_count,game_events
    

#data_path = os.path.join('D:\\tmp\\ml-100k', 'u.data')
#user_count, item_count, user_ratings = load_data(data_path)
game_count,event_count,game_events_dict = load_data(DATA_DIR+DATA_FOR_BPR)

game_count: 1108
event_count: 11189


In [4]:
#
def generate_test(game_events_dict):
    user_test = dict()
    for u, i_list in game_events_dict.items():
        user_test[u] = random.sample(game_events_dict[u], 1)[0]
    return user_test

temp_dict = generate_test(game_events_dict)

In [50]:
#generate dataset for training
def generate_train_batch(game_events_dict, temp_dict, event_count, batch_size):
    t = []
    
    #print('generate batch..')
    for b in range(batch_size):
        break_flag1 = 0
        break_flag2 = 0
        u = random.sample(game_events_dict.keys(), 1)[0]
        #print('u:',u)
        i = random.sample(game_events_dict[u], 1)[0]
        while i == temp_dict[u]:
            break_flag1 += 1
            i = random.sample(game_events_dict[u], 1)[0]
            if break_flag1 == 20:
                #game_events_dict.pop('u',None)
                #temp_dict.pop('u',None)
                break
            #print('i:',i)
        j = random.randint(1, event_count)
        while j in game_events_dict[u]:
            break_flag2 += 1
            j = random.randint(1, event_count)
            if break_flag2 == 20:
                #game_events_dict.pop('u',None)
                #temp_dict.pop('u',None)
                break
            #print('j:',j)
        t.append([u, i, j])
        #print(u,i,j)
    #print('generate batch done!')
    return numpy.asarray(t)

#train_set = generate_train_batch(game_events_dict,temp_dict,event_count)

In [61]:
def generate_test_batch(game_events_dict, temp_dict, event_count):
    for u in game_events_dict.keys():
        t = []
        i = temp_dict[u]
        #for j in range(1, event_count+1):
        for j in range(1, 1001):
            if not (j in game_events_dict[u]):
                t.append([u, i, j])
        yield numpy.asarray(t)

In [46]:
def bpr_mf(game_count, event_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [game_count+1, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [event_count+1, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
    
    # MF predict: u_i > u_j
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keepdims=True)
    
    # AUC for one user:
    # reasonable iff all (u,i,j) pairs are from the same user
    # 
    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    
    l2_norm = tf.add_n([
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

In [84]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(game_count, event_count, 20)
#     print(u, i, j, mf_auc, bprloss, train_op)
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    print('start epoch')
    for epoch in range(1, 2):
        print ("epoch: ", epoch)
        _batch_bprloss = 0
        for k in tqdm(range(1, 5000)): # uniform samples from training set
            uij = generate_train_batch(game_events_dict, temp_dict,event_count,batch_size=512)

            _bprloss, _train_op = session.run([bprloss, train_op], 
                                feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        
        print ("bpr_loss: ", _batch_bprloss / k)
        print ("_train_op")

        games_count = 0
        _auc_sum = 0.0
        test_bprloss = 0.0
        # each batch will return only one user's auc
        for t_uij in generate_test_batch(game_events_dict, temp_dict, event_count):

            _auc, _test_bprloss = session.run([mf_auc, bprloss],
                                    feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]}
                                )
            games_count += 1
            _auc_sum += _auc
            test_bprloss += _test_bprloss
#            print ("test_loss: ", test_bprloss/games_count)
        print ("test_loss: ", test_bprloss/games_count, "test_auc: ", _auc_sum/games_count)
        print ("")
    variable_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variable_names)
    saver.save(session, SAVE_DIR)
    for k,v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)
print('all done')

  0%|          | 1/4999 [00:00<08:45,  9.50it/s]

start epoch
epoch:  1


100%|██████████| 4999/4999 [01:23<00:00, 59.59it/s]


bpr_loss:  0.7229769994888717
_train_op
test_loss:  0.7489471516239083 test_auc:  0.5111525780014906

Variable:  user_emb_w:0
Shape:  (1109, 20)
[[ 0.03679958 -0.09989663  0.02503717 ... -0.07627834 -0.0605009
  -0.03673812]
 [-0.04986278 -0.09031277 -0.0500199  ... -0.19825013  0.12151904
   0.10557431]
 [ 0.00365524  0.14824083 -0.03527503 ...  0.01671001  0.11243264
   0.06172235]
 ...
 [-0.00991808  0.08441954 -0.02329948 ... -0.17994705 -0.13385957
  -0.0119026 ]
 [-0.06867337  0.03956933  0.06494844 ... -0.01831766  0.09859017
  -0.16752434]
 [-0.16004193 -0.14453687 -0.03481292 ...  0.08316372  0.04111993
   0.05761187]]
Variable:  item_emb_w:0
Shape:  (11190, 20)
[[-0.19654194  0.09346362 -0.15887265 ...  0.04635011  0.04819652
   0.00392018]
 [ 0.1684987  -0.05777324  0.05834768 ... -0.03552048 -0.12857795
  -0.04508829]
 [ 0.02511788  0.19203483 -0.05736518 ... -0.09844682 -0.09532938
  -0.10834677]
 ...
 [-0.05310225 -0.15992926 -0.16693182 ...  0.13361815  0.03638485
  -0.0

In [58]:
print(len(t_uij))
print(len(uij))

11110
50


In [29]:
_test_bprloss

1.1672156

In [ ]:
saver.save(session, SAVE_DIR)

In [77]:
session1 = tf.Session()
u1_dim = tf.expand_dims(values[0][1], 0)
u1_all = tf.matmul(u1_dim, values[1],transpose_b=True)
result_1 = session1.run(u1_all)
print (result_1)

[[ 0.16464396  0.00183702 -0.02877046 ... -0.05436823  0.05229869
  -0.07270078]]


In [83]:
print("new recommedation ：")
p = numpy.squeeze(result_1)
p[numpy.argsort(p)[:-10]] = 0
for index in range(len(p)):
    if p[index] != 0:
        select = data[data['new_event_id']==index].drop_duplicates('eventname')
  
        print (index, p[index],select['eventname'])

new recommedation ：
569 0.18474106 51023    black_market_buy_toy
Name: eventname, dtype: object
1090 0.20001167 78596    systemNetworkSlowImage
Name: eventname, dtype: object
2162 0.17544779 192626    enterMapView
Name: eventname, dtype: object
3380 0.17380595 299303    nameChanged
Name: eventname, dtype: object
5957 0.19109626 482335    starterPackPopupDisplayed
Name: eventname, dtype: object
6394 0.1952556 514548    beebreaker_activity_start
Name: eventname, dtype: object
6450 0.19272463 520913    addCrystal
Name: eventname, dtype: object
7890 0.17444888 642755    enemykilled
Name: eventname, dtype: object
8743 0.20903704 683869    appsFlyerConversionDataError
Name: eventname, dtype: object
10932 0.18367468 850277    hintMenuOpened
Name: eventname, dtype: object


In [65]:
data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [67]:
data[data['new_event_id']==11118]
data[data['new_game_id']==1].drop_duplicates('eventname')

,gameid,genre,platform,eventname,eventparameter,eventtype,new_game_id,new_event_id
0,3,No genre,ios,transaction,focus,custom,1,1
1,3,No genre,ios,gameEnded,attack,custom,1,2
2,3,No genre,ios,missionFailed,attack,custom,1,3
3,3,No genre,ios,newPlayer,attack,standard,1,4
4,3,No genre,ios,levelUp,attack,custom,1,5
5,3,No genre,ios,characterCreated,attack,custom,1,6
6,3,No genre,ios,guild,attack,custom,1,7
7,3,No genre,ios,social,attack,custom,1,8
9,3,No genre,ios,clientDevice,attack,standard,1,9
10,3,No genre,ios,missionCompleted,attack,custom,1,10
